<a href="https://colab.research.google.com/github/Tclack88/AB-Demo/blob/master/csv_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [263]:
!rm * && rm -r sample_data

rm: cannot remove 'sample_data': Is a directory


In [264]:
!ls

sample_data


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import requests
from multiprocessing.dummy import Pool
import os
from bs4 import BeautifulSoup

In [0]:
### Supporting Definitions ###

# get user_data links
def get_csv_url(id):
  user = requests.head(f'https://www.mountainproject.com/user/{id}').headers['Location'].split('/')[-1]
  return f'https://www.mountainproject.com/user/{id}/{user}/tick-export'
  


# add danger column
danger_rating = {'PG13':1 ,'R':2,'X':3}
def get_danger(grade):
  try:
    danger = danger_rating[grade.split()[1]]
    return danger
  except:
    return 0 

  
# A possible arrangement of difficulties
# 10a 10- 10a/b 10b 10 10b/c 10c 10+ 10c/d 10d 11a  --> 10.0, 10.1, ... , 10.9, 11

grade_chart = pd.read_html("https://www.mountainproject.com/international-climbing-grades")

old_ropes = grade_chart[0].YDSUSA[:-1].to_list()
new_ropes = [0,0,0,0,1,2,3,4,5,6,7,7.5,8,8.5,8.8,9,9.5,9.8]+list(np.round(np.arange(10,16,.1),1))
ropes_convert = dict(zip(old_ropes,new_ropes))

old_boulders = grade_chart[1].HuecoUSA[:-1].to_list()
new_boulders = [0,0,0,0,0]+list(np.round(np.arange(1,17.5,.25),1))

boulders_convert = dict(zip(old_boulders,new_boulders))

def clean_grade(grade):
  grade = str(grade).split()[0]
  if grade[0] == '5':
    grade = ropes_convert[grade]
    return grade
  elif grade[0] == 'V':
    grade = boulders_convert[grade]
    return grade
  
  

# create user ticks
def create_user_ticks(file):
  ticks = pd.read_csv(file)
  cols = ['Date','Rating','Pitches','Style','Lead Style','Route Type','Location']
  ticks = ticks[cols]

  #rename cols
  old_names = ticks.columns.to_list()
  new_names = ['date','grade','pitches','style','lead_style','type','location']
  rename_cols = dict(zip(old_names,new_names))
  ticks = ticks.rename(columns=rename_cols)

  # date to datetime
  ticks.date = pd.to_datetime(ticks.date)

  # add danger column
  ticks['danger'] = ticks.grade.apply(get_danger)

  
  ticks.grade = ticks.grade.apply(clean_grade)
  return ticks

In [0]:
# Main Functions

def create_climber_stats(ticks):
  climber_dict = {}
  climber_dict.update(years_total = [ticks.date.max()-ticks.date.min()],
                     climbs_total = [ticks.shape[0]],
                     pitches_total = [ticks.pitches.sum()],
                     route_mean = [ticks[ticks['type'] != 'Boulder'].grade.mean()],
                     route_max = [ticks[ticks['type'] != 'Boulder'].grade.max()],
                     boulder_mean = [ticks[ticks.type == 'Boulder'].grade.mean()],
                     boulder_max = [ticks[ticks.type == 'Boulder'].grade.max()],
                     danger_factor = [ticks.danger.mean()],
                     solos = [ticks[ticks['style'] == 'Solo'].shape[0]],
                     hardest_solo = [ticks[ticks['style'] == 'Solo'].grade.max()],
                     trad_count = [ticks.type.apply(lambda x: 'Trad' in x).sum()],
                     sport_count = [ticks.type.apply(lambda x: 'Sport' in x).sum()],
                     locations = [ticks.location.nunique()],
                     success = [(ticks.lead_style=='Onsight').sum()])
                     # trevor['Route Type'].apply(lambda x: 'Trad' in x).sum()
                     # needs to be list when entries are a scalar
  climber_stats = pd.DataFrame(climber_dict)
  return climber_stats



def extract_information(csv_urls):
  for url in csv_urls:    # Download user csv's
    command = "wget "+url
    os.system(command)
  files = [f for f in os.listdir('.') if os.path.isfile(f)]
  #climber_chunk_stats = pd.DataFrame(columns = ['years_climbing','climbs_total','pitches_total','route_average','route_max'])
  climber_chunk_stats = pd.DataFrame()
  
  for file in files:
    try:
      ticks = create_user_ticks(file)
      if ticks.shape[0] == 0:
        continue
      climber_stats = create_climber_stats(ticks)
    #print(climber_stats)
      climber_chunk_stats = pd.concat([climber_chunk_stats,climber_stats])
    except:
      continue
  return climber_chunk_stats

In [0]:
# Grab names to seed the search (most popular male and female names from 1985)
name_url = "https://www.weddingvendors.com/baby-names/popular/1985/"

source = requests.get(name_url).text
soup = BeautifulSoup(source,'html.parser')
names = soup.find_all('td', class_='n')
possible_users = []
for name in names:
  possible_user = name.text
  possible_users.append(possible_user)


In [0]:
possible_users = ['Jake']
url_start = "https://www.mountainproject.com/ajax/public/search/results/category?q="
url_end = "&c=Users&o=0&s=Default"

## WARNING: This next cell will take a while

In [270]:
p = Pool(12)

#climber_stats = pd.DataFrame(columns = ['years_climbing','climbs_total','pitches_total','route_average','route_max'])
climber_stats = pd.DataFrame()
for name in possible_users:
  query = url_start+name+url_end
  resp = requests.get(query)
  users = resp.json()['results']['Users']
  user_ids = [u[12:21] for u in users]
  csv_urls = p.map(get_csv_url,user_ids)
  climber_chunk_stats = extract_information(csv_urls)
  climber_stats = pd.concat([climber_stats,climber_chunk_stats])
  os.system('rm * -f') # clear the latest chunk of downloaded csv's to minimize space usage

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [271]:
!ls

sample_data


In [272]:
climber_stats

,years_total,climbs_total,pitches_total,route_mean,route_max,boulder_mean,boulder_max,danger_factor,solos,hardest_solo,trad_count,sport_count,locations,success
0,0 days,9,9,9.577778,10.5,NaN,NaN,0.00,0,NaN,0,9,5,0
0,32 days,20,20,9.125000,10.3,1.35625,3.2,0.05,1,3.2,0,4,10,3
0,0 days,5,5,8.620000,10.3,NaN,NaN,0.00,0,NaN,0,5,2,2
0,0 days,1,1,10.300000,10.3,NaN,NaN,1.00,0,NaN,0,0,1,0


In [0]:
climber_stats.to_csv('climber_data.csv')

In [274]:
!cat climber_data.csv

,years_total,climbs_total,pitches_total,route_mean,route_max,boulder_mean,boulder_max,danger_factor,solos,hardest_solo,trad_count,sport_count,locations,success
0,0 days 00:00:00.000000000,9,9,9.577777777777776,10.5,,,0.0,0,,0,9,5,0
0,32 days 00:00:00.000000000,20,20,9.125,10.3,1.35625,3.2,0.05,1,3.2,0,4,10,3
0,0 days 00:00:00.000000000,5,5,8.62,10.3,,,0.0,0,,0,5,2,2
0,0 days 00:00:00.000000000,1,1,10.3,10.3,,,1.0,0,,0,0,1,0


## Old Stuff Below

In [39]:
url = "https://www.mountainproject.com/user/109791883/trevor-clack/tick-export"
url = "https://www.mountainproject.com/user/106027958/brad-g/tick-export"
trevor = pd.read_csv(url)
trevor.head()

,Date,Route,Rating,Notes,URL,Pitches,Location,Avg Stars,Your Stars,Style,Lead Style,Route Type,Your Rating,Length,Rating Code
0,2018-04-03,Fairy Tales,5.13-,Onsight,https://www.mountainproject.com/route/10637253...,1,Utah > Moab Area > Indian Creek > Pistol Whipped,3.1,-1,Lead,NaN,Trad,NaN,60.0,8700
1,2018-04-02,Savelli Crack,5.13b/c PG13,NaN,https://www.mountainproject.com/route/10825826...,1,Utah > Moab Area > Indian Creek > Supercrack B...,4.0,-1,NaN,NaN,Trad,NaN,80.0,9100
2,2018-03-28,International Affair,5.12,Onsight,https://www.mountainproject.com/route/10601765...,1,Utah > Moab Area > Indian Creek > Supercrack B...,3.3,-1,NaN,NaN,Trad,NaN,50.0,7000
3,2018-03-27,The Montana Weed Connection,5.13-,NaN,https://www.mountainproject.com/route/10698761...,1,Utah > Moab Area > Indian Creek > Pistol Whipped,3.0,-1,NaN,NaN,Trad,NaN,35.0,8700
4,2018-03-22,Carbondale Short Bus,5.13+ R,NaN,https://www.mountainproject.com/route/10800630...,1,Utah > Moab Area > Indian Creek > 4X4,4.0,-1,Lead,NaN,Trad,NaN,60.0,9300


In [40]:
ticks = create_user_ticks(url)
ticks.head()

,date,grade,pitches,style,lead_style,type,location,danger
0,2018-04-03,13.1,1,Lead,NaN,Trad,Utah > Moab Area > Indian Creek > Pistol Whipped,0
1,2018-04-02,13.5,1,NaN,NaN,Trad,Utah > Moab Area > Indian Creek > Supercrack B...,1
2,2018-03-28,12.4,1,NaN,NaN,Trad,Utah > Moab Area > Indian Creek > Supercrack B...,0
3,2018-03-27,13.1,1,NaN,NaN,Trad,Utah > Moab Area > Indian Creek > Pistol Whipped,0
4,2018-03-22,13.7,1,Lead,NaN,Trad,Utah > Moab Area > Indian Creek > 4X4,2


In [42]:
n = 5  
fracs = (np.linspace(0,ticks.shape[0],n)).astype(int)
fracs
for i in range(n-1):
  tick_range = ticks.iloc[fracs[i]:fracs[i+1],1]
  print(tick_range.max())
# begin = ticks.date.min()
# for i in range n:
#   print(ticks[ticks.date.loc

#np.array([0,.25,.5,.27,1])
#time_fracs = time_fracs*(ticks.date.max() - ticks.date.min())
#for i,frac in time_fracs:
#for i in time_fracs:
#  print(ticks[ticks.date

14.0
13.9
13.6
13.1


In [8]:
!ls

sample_data


In [0]:
# empty = pd.DataFrame(columns=['date','grade','pitches','style','lead_style','type','danger'])
# #empty.merge(ticks,how='outer')
# pd.concat([empty,ticks])
# empty.head(5)
# ticks.head(5)


In [0]:
# users = resp.json()['results']['Users']
# users[0:5]

In [0]:
# user_ids = [u[12:21] for u in users]

In [0]:
# from multiprocessing.dummy import Pool
# p = Pool(12)
# def get_csv_url(id):
#   user = requests.head(f'https://www.mountainproject.com/user/{id}').headers['Location'].split('/')[-1]
#   return f'https://www.mountainproject.com/user/{id}/{user}/tick-export'
# csv_urls = p.map(get_csv_url, user_ids)

In [0]:
# csv_urls

In [0]:
# import os
# for url in csv_urls:
#   command = "wget "+url
#   os.system(command)

In [0]:
#!rm -r sample_data && ls
#!rm * && ls

In [0]:
# Morgan's Routes
#command1 = "wget https://www.mountainproject.com/user/110296901/morgan-f/tick-export"
# My Routes
#command2 = "wget https://www.mountainproject.com/user/109791883/trevor-clack/tick-export"
# Brad G.
#command3 = "wget https://www.mountainproject.com/user/106027958/brad-g/tick-export"
#Ryan Murphy
#command4 = "wget https://www.mountainproject.com/user/106234022/stormeh/tick-export"
#os.system(command1)
#os.system(command2)
#os.system(command3)
#os.system(command4)

In [0]:
# files = [f for f in os.listdir('.') if os.path.isfile(f)]
# files
# for file in files:
#   ticks = pd.read_csv('file')


In [0]:
# # add danger column
# danger_rating = {'PG13':1 ,'R':2,'X':3}
# def get_danger(grade):
#   try:
#     danger = danger_rating[grade.split()[1]]
#     return danger
#   except:
#     return 0 

  
# # A possible arrangement of difficulties
# # 10a 10- 10a/b 10b 10 10b/c 10c 10+ 10c/d 10d 11a  --> 10.0, 10.1, ... , 10.9, 11

# grade_chart = pd.read_html("https://www.mountainproject.com/international-climbing-grades")

# old_ropes = grade_chart[0].YDSUSA[:-1].to_list()
# new_ropes = [0,0,0,0,1,2,3,4,5,6,7,7.5,8,8.5,8.8,9,9.5,9.8]+list(np.round(np.arange(10,16,.1),1))
# ropes_convert = dict(zip(old_ropes,new_ropes))

# old_boulders = grade_chart[1].HuecoUSA[:-1].to_list()
# new_boulders = [0,0,0,0,0]+list(np.round(np.arange(1,17.5,.25),1))

# boulders_convert = dict(zip(old_boulders,new_boulders))

# def clean_grade(grade):
#   grade = grade.split()[0]
#   if grade[0] == '5':
#     grade = ropes_convert[grade]
#     return grade
#   elif grade[0] == 'V':
#     grade = boulders_convert[grade]
#     return grade
  
  

# # create user ticks
# def create_user_ticks(file):
#   ticks = pd.read_csv(file)
#   cols = ['Date','Rating','Pitches','Style','Lead Style','Route Type']
#   ticks = ticks[cols]

#   #rename cols
#   old_names = ticks.columns.to_list()
#   new_names = ['date','grade','pitches','style','lead_style','type']
#   rename_cols = dict(zip(old_names,new_names))
#   ticks = ticks.rename(columns=rename_cols)

#   # date to datetime
#   ticks.date = pd.to_datetime(ticks.date)

#   # add danger column
#   ticks['danger'] = ticks.grade.apply(get_danger)

  
#   ticks.grade = ticks.grade.apply(clean_grade)
# ticks.head(30)


In [0]:
# A possible arrangement of difficulties
# 10a 10- 10a/b 10b 10 10b/c 10c 10+ 10c/d 10d 11a  --> 10.0, 10.1, ... , 10.9, 11

# grade_chart = pd.read_html("https://www.mountainproject.com/international-climbing-grades")

# old_ropes = grade_chart[0].YDSUSA[:-1].to_list()
# new_ropes = [0,0,0,0,1,2,3,4,5,6,7,7.5,8,8.5,8.8,9,9.5,9.8]+list(np.round(np.arange(10,16,.1),1))
# ropes_convert = dict(zip(old_ropes,new_ropes))

# old_boulders = grade_chart[1].HuecoUSA[:-1].to_list()
# new_boulders = [0,0,0,0,0]+list(np.round(np.arange(1,17.5,.25),1))

# boulders_convert = dict(zip(old_boulders,new_boulders))

# def clean_grade(grade):
#   grade = grade.split()[0]
#   if grade[0] == '5':
#     grade = ropes_convert[grade]
#     return grade
#   elif grade[0] == 'V':
#     grade = boulders_convert[grade]
#     return grade
    
# ticks.grade = ticks.grade.apply(clean_grade)

In [0]:
#tick_stats = 

In [0]:
# monthly_tick_average = ticks[ticks.type != 'Boulder'][['date','grade']].set_index('date').resample('M',how='mean').dropna()
#monthly_tick_average['max'] = ticks[ticks.lead_style == 'Onsight'][['date','grade']].set_index('date').resample('M',how='max').dropna().grade
#monthly_tick_average = monthly_tick_average.reset_index()

In [0]:
#Individual plotting of Average Grades
# plt.style.use('fivethirtyeight')
# monthly_tick_average.plot(legend=False,color='g')
# plt.title("Ryan Average Monthly grade (routes)",);

In [0]:
# lead = ticks[(ticks['style'] == 'Lead') & (ticks.lead_style =='Onsight')]
# high_grades = lead[lead.grade >= 10]
# high_grades.grade.value_counts().sort_index()